In [8]:
 !pip install sqlalchemy pymysql -q

In [9]:
 !pip install sqlalchemy sql_magic -q

In [10]:
%load_ext sql_magic

/opt/conda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [11]:
%sql

UsageError: Line magic function `%sql` not found.


In [12]:
import pandas as pd
import numpy as np


## Write pandas df to database

In [13]:
user="cornell"
password="Pitney@1a"
host="gec-dwh.ctz5iw1v2fh7.us-west-2.rds.amazonaws.com"
db="cornell"

conn = f"mysql+pymysql://{user}:{password}@{host}/{db}"

In [14]:
from sqlalchemy import create_engine

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

In [15]:
from pandas.io import sql


## Query database

In [9]:
result = engine.execute('show databases')
for _r in result:
    print(_r)

('information_schema',)
('cornell',)


In [10]:
result = engine.execute('show tables in cornell')
for _r in result:
    print(_r)

('deliver_package_info',)
('input_df',)
('ne_delivery_data_by_day_2018',)
('ne_returns_data_by_day_2018',)
('return_package_info',)
('return_package_info_update',)
('test_df',)


In [ ]:
result=engine.execute('select *, count(*) from cornell.ne_delivery_date_by_day_2018 group by originfacilityzip,destinationfacilityzip \
                        order by origindate ')


## Query first 10 delivery package data & fields name

In [61]:
result = engine.execute('select * from cornell.ne_delivery_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

(datetime.date(2018, 9, 30), 'Delivery', '362970510', '0.1950', 'Client - 2462', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082836', '0.2300', 'Client - 1216', '11096', 'Courier - Plainville', '2762')
(datetime.date(2018, 8, 14), 'Delivery', '355433187', '1.6600', 'Client - 1216', '11096', 'Courier - Secaucus', '7094')
(datetime.date(2018, 7, 11), 'Delivery', '350299758', '0.8800', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082834', '0.4850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082830', '15.1200', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 9), 'Delivery', '350140996', '1.3400', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 11), 'Delivery', '350286964', '0.1850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(20

In [59]:
result = engine.execute('describe cornell.deliver_package_info')
for _r in result:
    print(_r)

('index', 'bigint(20)', 'YES', 'MUL', None, '')
('origindate', 'date', 'YES', '', None, '')
('originfacilityzip', 'text', 'YES', '', None, '')
('destinationfacilityzip', 'text', 'YES', '', None, '')
('count', 'bigint(20)', 'YES', '', None, '')


## How many delivery data do we have

In [11]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018')
for _r in result:
    print(_r)

(61916290,)


## Daily demand of delivery packages

In [61]:
result = engine.execute('select origindate, count(*) from cornell.ne_delivery_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

KeyboardInterrupt: 

### The number of deliver packages from customer warehouse to USPS & store in a new table "deliver_package_info"

In [23]:
# The # of packages from customer warehouse to USPS 
deliver_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_delivery_data_by_day_2018 \
                        group by origindate,originfacilityzip,destinationfacilityzip')
deliver_df = pd.DataFrame(deliver_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])


In [24]:
deliver_df


,origindate,originfacilityzip,destinationfacilityzip,count
0,2017-12-31,17055,,56
1,2017-12-31,30349,,33
2,2018-01-01,06492,7094,1
3,2018-01-01,10974,01152,2
4,2018-01-01,10974,01546,9
5,2018-01-01,10974,01889,2
6,2018-01-01,10974,02205,1
7,2018-01-01,10974,02904,1
8,2018-01-01,10974,04074,5
9,2018-01-01,10974,07097,1


In [25]:
deliver_df.to_sql(con=engine, name='deliver_package_info', 
                if_exists='replace')

## Query first 10 return package data & fields name

In [54]:
result = engine.execute('select * from cornell.ne_returns_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

(datetime.date(2019, 1, 10), 'Return', 420456129, 0.1168, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2019, 1, 7), 'Return', 420156653, 3.35, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2019, 1, 7), 'Return', 420161921, 0.9369, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2019, 1, 7), 'Return', 420162311, 0.2733, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2019, 1, 7), 'Return', 420162321, 0.6128, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2019, 1, 7), 'Return', 420162133, 0.3108, 'IND - PartnerFacility', '46037', 'IND - Client', '46037')
(datetime.date(2018, 5, 25), 'Return', 394568185, 1.7835, 'ATL - PartnerFacility', '30349', 'Client - 52909', '92407')
(datetime.date(2018, 8, 27), 'Return', 405004216, 5.105, 'IND - PartnerFacility', '46037', 'Client - 52909', '92407')
(datetime.date(2018, 8, 27), 'Return', 404860180, 10.68, 'IND - Part

In [55]:
result = engine.execute('describe cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

('origindate', 'date', 'YES', '', None, '')
('packagetype', 'varchar(50)', 'YES', '', None, '')
('packagekey', 'bigint(20)', 'YES', '', None, '')
('actualweight', 'double', 'YES', '', None, '')
('originfacilitymodified', 'varchar(50)', 'YES', '', None, '')
('originfacilityzip', 'varchar(50)', 'YES', '', None, '')
('destinationfacility', 'varchar(50)', 'YES', '', None, '')
('destinationfacilityzip', 'varchar(50)', 'YES', '', None, '')


## How many return data do we have

In [9]:
result = engine.execute('select count(*) from cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

(33276614,)


## Daily demand of return packages

In [ ]:
result = engine.execute('select origindate, count(*) from cornell.ne_returns_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

### The number of daily return packages from USPS to customer warehouse & store in a new table "return_package_info"

In [16]:
return_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_returns_data_by_day_2018 \
                        group by origindate,originfacilityzip,destinationfacilityzip ')
return_df = pd.DataFrame(return_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])




In [17]:
return_df.to_sql(con=engine, name='return_package_info', if_exists='replace')

In [18]:
return_df

,origindate,originfacilityzip,destinationfacilityzip,count
0,2017-12-31,01152,08817,1
1,2017-12-31,01152,17527,1
2,2017-12-31,01152,17699,1
3,2017-12-31,01152,27801,1
4,2017-12-31,01889,01460,1
5,2017-12-31,01889,08817,1
6,2017-12-31,01889,08831,1
7,2017-12-31,01889,08837,1
8,2017-12-31,01889,11096,1
9,2017-12-31,01889,16368,1


## geodecode zip3

#### zip3 reference dict

In [19]:
zip3_dic={}
zip3_file='/home/fusion/jupyterhub-notebooks/Cornell_PB_NE_Hub/references/zip3_centroids.csv'
import csv
with open(zip3_file,'r',encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        zip3_dic[row[2]]=[row[3],row[4],row[5]]


### decodezip3 function

In [20]:
def decodezip3(zip3,reference):
    res=[]
    for i in zip3:
        if len(i)==3:
            i='00'+i
        elif len(i)==4:
            i='0'+i
            
        zipp=i[0:3]
        if not i:
            tmp=[zipp,'','','']
        else:
            if zipp in reference.keys():
                zipinfo=reference[zipp]
                tmp=[zipp,zipinfo[0],zipinfo[1],zipinfo[2]]
            else:
                tmp=[zipp,'','','']    
        res.append(tmp)
    return res


### decode deliver - origin

In [17]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from deliver_package_info')

In [18]:
deliver_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [26]:
deliver_tmp=decodezip3(deliver_df['originfacilityzip'],zip3_dic)

In [27]:
tmp_df=pd.DataFrame(deliver_tmp,columns=['origin_zip3','origin_state','origin_lat','origin_lng'])

In [28]:
deliver_df=deliver_df.join(tmp_df)

In [29]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng
0,2017-12-31,17055,,56,170,PA,40.5266561258918,-77.8269602130124
1,2017-12-31,30349,,33,303,GA,33.7554224110405,-84.3904423185221
2,2018-01-01,06492,7094,1,064,CT,41.3447619004598,-73.068704647852
3,2018-01-01,10974,01152,2,109,NY,41.1434616274125,-74.1238460876456
4,2018-01-01,10974,01546,9,109,NY,41.1434616274125,-74.1238460876456
5,2018-01-01,10974,01889,2,109,NY,41.1434616274125,-74.1238460876456
6,2018-01-01,10974,02205,1,109,NY,41.1434616274125,-74.1238460876456
7,2018-01-01,10974,02904,1,109,NY,41.1434616274125,-74.1238460876456
8,2018-01-01,10974,04074,5,109,NY,41.1434616274125,-74.1238460876456
9,2018-01-01,10974,07097,1,109,NY,41.1434616274125,-74.1238460876456


### decode deliver - dest

In [30]:
destination_tmp=decodezip3(deliver_df['destinationfacilityzip'],zip3_dic)

In [31]:
dest_df=pd.DataFrame(destination_tmp,columns=['dest_zip3','dest_state','dest_lat','dest_lng'])


In [32]:
deliver_df=deliver_df.join(dest_df)

In [33]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng,dest_zip3,dest_state,dest_lat,dest_lng
0,2017-12-31,17055,,56,170,PA,40.5266561258918,-77.8269602130124,,,,
1,2017-12-31,30349,,33,303,GA,33.7554224110405,-84.3904423185221,,,,
2,2018-01-01,06492,7094,1,064,CT,41.3447619004598,-73.068704647852,070,NJ,40.5833124911422,-74.2703150322948
3,2018-01-01,10974,01152,2,109,NY,41.1434616274125,-74.1238460876456,011,MA,42.1056970748294,-72.5980788159072
4,2018-01-01,10974,01546,9,109,NY,41.1434616274125,-74.1238460876456,015,MA,42.1968723825208,-71.8453871667191
5,2018-01-01,10974,01889,2,109,NY,41.1434616274125,-74.1238460876456,018,MA,42.485950411217,-71.1553696696814
6,2018-01-01,10974,02205,1,109,NY,41.1434616274125,-74.1238460876456,022,MA,42.3582224850777,-71.0584272136915
7,2018-01-01,10974,02904,1,109,NY,41.1434616274125,-74.1238460876456,029,RI,41.8258558538419,-71.4134288124013
8,2018-01-01,10974,04074,5,109,NY,41.1434616274125,-74.1238460876456,040,ME,43.530478933588,-70.9212281508382
9,2018-01-01,10974,07097,1,109,NY,41.1434616274125,-74.1238460876456,070,NJ,40.5833124911422,-74.2703150322948


In [34]:

deliver_df.to_sql(con=engine, name='deliver_package_info',
                  if_exists='replace')
 

### decode return origin

In [21]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from return_package_info')
return_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])



In [22]:
return_tmp=decodezip3(return_df['originfacilityzip'],zip3_dic)

In [23]:
tmp_df=pd.DataFrame(return_tmp,columns=['origin_zip3','origin_state','origin_lat','origin_lng'])

In [24]:
return_df=return_df.join(tmp_df)


In [40]:
return_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng
0,2018-01-02,15930,27801,2,159,PA,40.3278437185358,-78.9138177538114
1,2018-01-02,23222,07836,1,232,VA,37.5392326292499,-77.4374487131616
2,2018-01-02,4092,07836,4,040,ME,43.530478933588,-70.9212281508382
3,2018-01-02,01472,52404,1,014,MA,42.5853946189292,-71.8170603313981
4,2018-01-02,15930,17699,2,159,PA,40.3278437185358,-78.9138177538114
5,2018-01-02,20676,21704,1,206,MD,38.6176487130023,-76.8552682490031
6,2018-01-02,08880,16368,1,088,NJ,40.6240218011197,-74.8843774239638
7,2018-01-02,20676,46037,1,206,MD,38.6176487130023,-76.8552682490031
8,2018-01-02,07721,21704,1,077,NJ,40.3606127258978,-74.0780750505036
9,2018-01-02,15930,30349,1,159,PA,40.3278437185358,-78.9138177538114


### decode return destination

In [25]:
return_tmp=decodezip3(return_df['destinationfacilityzip'],zip3_dic)

In [26]:
tmp_df=pd.DataFrame(return_tmp,columns=['dest_zip3','dest_state','dest_lat','dest_lng'])

In [27]:
return_df=return_df.join(tmp_df)

In [28]:
return_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_zip3,origin_state,origin_lat,origin_lng,dest_zip3,dest_state,dest_lat,dest_lng
0,2017-12-31,01152,08817,1,011,MA,42.1056970748294,-72.5980788159072,088,NJ,40.6240218011197,-74.8843774239638
1,2017-12-31,01152,17527,1,011,MA,42.1056970748294,-72.5980788159072,175,PA,40.1567631818116,-76.2045168810913
2,2017-12-31,01152,17699,1,011,MA,42.1056970748294,-72.5980788159072,176,PA,40.0730558106315,-76.3141886839793
3,2017-12-31,01152,27801,1,011,MA,42.1056970748294,-72.5980788159072,278,NC,35.911301017443,-77.7362497363615
4,2017-12-31,01889,01460,1,018,MA,42.485950411217,-71.1553696696814,014,MA,42.5853946189292,-71.8170603313981
5,2017-12-31,01889,08817,1,018,MA,42.485950411217,-71.1553696696814,088,NJ,40.6240218011197,-74.8843774239638
6,2017-12-31,01889,08831,1,018,MA,42.485950411217,-71.1553696696814,088,NJ,40.6240218011197,-74.8843774239638
7,2017-12-31,01889,08837,1,018,MA,42.485950411217,-71.1553696696814,088,NJ,40.6240218011197,-74.8843774239638
8,2017-12-31,01889,11096,1,018,MA,42.485950411217,-71.1553696696814,110,NY,40.7220752067917,-73.7068341419121
9,2017-12-31,01889,16368,1,018,MA,42.485950411217,-71.1553696696814,163,PA,41.4542864255809,-79.6519755218344


In [29]:
# return_df.to_sql(con=engine, name='return_package_info', if_exists='replace',dtype=['datetime.date','text','text','bigint(20)','text','text','text','double','double','text','text','text','double','double'])
return_df.to_sql(con=engine, name='return_package_info', if_exists='replace')


## Detect zip code which cannot be decode

In [5]:
zip3_dic={}
zip3_file='/home/fusion/jupyterhub-notebooks/Cornell_PB_NE_Hub/references/zip3_centroids.csv'
import csv
with open(zip3_file,'r',encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        zip3_dic[row[2]]=[row[3],row[4],row[5]]

In [6]:
def decodezip3(zip3,reference):
    res=[]
    for i in zip3:
        if len(i)==3:
            i='00'+i
        elif len(i)==4:
            i='0'+i
            
        zipp=i[0:3]
        if not i:
            tmp=[zipp,'','','']
        else:
            if zipp in reference.keys():
                zipinfo=reference[zipp]
                tmp=[zipp,zipinfo[0],zipinfo[1],zipinfo[2]]
            else:
                print(zipp)
                tmp=[zipp,'','','']    
        res.append(tmp)
    return res


In [13]:
r709=engine.execute('select originfacilityzip, destinationfacilityzip, count(*) from ne_returns_data_by_day_2018 where originfacilityzip REGEXP \'^709\' or destinationfacilityzip REGEXP \'^709\'')
for _r in r709:
    print(_r)

('7094', '92407', 1711093)
